In [9]:
18000/60/60



5.0

In [6]:
using LinearMaps, IterativeSolvers, LinearAlgebra, JLD2

function read_eigenresults(number::Int)
    output_file = "test/Eigenvalues_folder/eigenresults_matrix_$(number).jld2"
    println("Reading eigenvalues from $output_file")
    data = jldopen(output_file, "r")
    eigenvalues = -data["Σexact"]
    close(data)
    return sort(eigenvalues)
end

eigenvalues_1 = read_eigenresults(1)
eigenvalues_2 = read_eigenresults(2)
eigenvalues_3 = read_eigenresults(3)
eigenvalues_4 = read_eigenresults(4)
eigenvalues_5 = read_eigenresults(5)
eigenvalues_6 = read_eigenresults(6)

# read first 20 eigenvalues
for i in 1:6
    println("First 20 eigenvalues of matrix $i: ", eval(Symbol("eigenvalues_$i"))[1:20])
end


Reading eigenvalues from test/Eigenvalues_folder/eigenresults_matrix_1.jld2
Reading eigenvalues from test/Eigenvalues_folder/eigenresults_matrix_2.jld2
Reading eigenvalues from test/Eigenvalues_folder/eigenresults_matrix_3.jld2
Reading eigenvalues from test/Eigenvalues_folder/eigenresults_matrix_4.jld2
Reading eigenvalues from test/Eigenvalues_folder/eigenresults_matrix_5.jld2
Reading eigenvalues from test/Eigenvalues_folder/eigenresults_matrix_6.jld2
First 20 eigenvalues of matrix 1: [-23.978377110229157, -19.657075544253022, -17.094688632871684, -15.293640659853793, -13.920120748628545, -12.820159581337812, -11.909846834111327, -11.138381170404044, -10.472712459877043, -9.890125928428331, -9.374396607931594, -8.91347271912299, -8.49825870747107, -8.121661770435725, -7.778045105263762, -7.462941986363561, -7.172618370252416, -6.904094822831796, -6.654794434123425, -6.42259819539194]
First 20 eigenvalues of matrix 2: [-23.978455365167758, -19.657172709731952, -17.094807897576505, -15.2

In [7]:
using IncompleteCholesky

ArgumentError: ArgumentError: Package IncompleteCholesky not found in current path.
- Run `import Pkg; Pkg.add("IncompleteCholesky")` to install the IncompleteCholesky package.

In [ ]:
using LinearMaps, IterativeSolvers, LinearAlgebra

# Dummy FLOP counters
global NFLOPs = 0
count_matmul_flops(args...) = nothing
count_vec_add_flops(args...) = nothing
count_vec_scaling_flops(args...) = nothing

# Your correction_equations_minres function here
function correction_equations_minres(A, U, lambdas, R; tol=1e-1, maxiter=100)
    global NFLOPs
    n, k = size(U)
    S = zeros(eltype(A), n, k)

    for j in 1:k
        λ, r = lambdas[j], R[:, j]

        M_apply = function(x)
            x_perp = x - (U * (U' * x)); 
            tmp = (A * x_perp) - λ * x_perp; 
            res = tmp - (U * (U' * tmp)); 
            return res
        end

        M_op = LinearMap{eltype(A)}(M_apply, n, n; ishermitian=true)

        rhs = r - (U * (U' * r)); 
        rhs = -rhs

        # Call MINRES with logging
        s_j, history = minres(M_op, rhs; reltol=tol, maxiter=maxiter, log=true)

        println("Vector $j: MINRES iterations = ", history.iters)

        s_j = s_j - (U * (U' * s_j)); 
        S[:, j] = s_j
    end
    return S
end

# === Example Problem ===
n = 10
A = Symmetric(randn(n, n))  # symmetric matrix
U = randn(n, 2)             # trial subspace
lambdas = [1.0, 2.0]        # fake Ritz values
R = randn(n, 2)             # fake residuals

S = correction_equations_minres(A, U, lambdas, R; tol=1e-6, maxiter=50)
println("Solution matrix S =")
println(S)


In [ ]:
using BenchmarkTools

function algo1(n)
    return sum(1:n)
end

function algo2(n)
    return n * (n + 1) ÷ 2
end

@btime algo1(10^6)
@btime algo2(10^6)


In [ ]:
using BenchmarkTools, Plots

function benchmark_scaling(f, sizes)
    times = Float64[]
    for n in sizes
        t = @belapsed f($n)   # high-precision elapsed time
        push!(times, t)
    end
    return times
end

sizes = 10 .^ (2:6)
times1 = benchmark_scaling(algo1, sizes)
times2 = benchmark_scaling(algo2, sizes)

plot(sizes, times1, label="algo1", xscale=:log10, yscale=:log10, xlabel="Input size n", ylabel="Time (s)")
plot!(sizes, times2, label="algo2")


In [ ]:
Profile.clear()
@profile algo1(10^6)
using ProfileView   # nice GUI flamegraphs
ProfileView.view()


In [ ]:
using PAPI

PAPI.start_counters(["PAPI_FP_OPS"]) do
    A = rand(1000,1000)
    B = rand(1000,1000)
    C = A * B
end


In [ ]:
using IterativeSolvers
using LinearAlgebra

# Example system: A * x = b
A = [4.0 1.0;
     1.0 3.0]
b = [1.0, 2.0]

# Solve with GMRES
x, history = gmres(A, b; log=true)

println("Solution x = ", x)
println("Residual norms = ", history.data)

# exact solution
x_exact = A \ b
println("Exact solution x = ", x_exact)